In [2]:
from PIL import Image
from whylogs.extras.image_metric import log_image
import os

training_path = '/home/vishesh/Desktop/datasets/ships-data/train_images'

# initialize reference_profile
reference_profile = None

# loop over each image in training folder
for i, filename in enumerate(os.listdir(training_path)[:1000]):
    if i % 100 == 0:
        print(i)
    
    file_path = os.path.join(training_path, filename)
    img = Image.open(file_path)
    profile = log_image(img).profile()
    profile_view = profile.view()

    # merge each profile while looping
    if reference_profile is None:
        reference_profile = profile_view
    else:
        reference_profile = reference_profile.merge(profile_view)

0
100
200
300
400
500
600
700
800
900


In [3]:
perturbation = "defocus_blur"
strength = 6


inference_path = f'/home/vishesh/Desktop/datasets/ships-data/test_images/varying/{perturbation}/{strength}'

inference_profile = None

# loop over each image in training folder
for i, filename in enumerate(os.listdir(inference_path)):
    if i % 100 == 0:
        print(i)
    
    file_path = os.path.join(inference_path, filename)
    img = Image.open(file_path)
    profile = log_image(img).profile()
    profile_view = profile.view()

    # merge each profile while looping
    if inference_profile is None:
        inference_profile = profile_view
    else:
        inference_profile = inference_profile.merge(profile_view)

0
100
200
300
400
500
600
700
800
900


In [4]:
from whylogs.viz.drift.column_drift_algorithms import calculate_drift_scores
from whylogs.migration.uncompound import _uncompound_dataset_profile

inference_profile = _uncompound_dataset_profile(inference_profile)
reference_profile = _uncompound_dataset_profile(reference_profile)

drift_values = calculate_drift_scores(
        inference_profile, reference_profile, drift_map=None, with_thresholds=True
    )


In [5]:
for key, drift_details in drift_values.items():
    if drift_details is None: continue
    
    drift_score = (1 - drift_details["pvalue"]) * drift_details["statistic"]
    drift_values[key]["drift_score"] = drift_score
    
    print(drift_score)
    

0.11799981148502028
0.10999889432860988
0.0
0.07232839441828914
0.008975911715264984
0.0
0.0
0.0869160450376772
0.007580499898628976
0.04836288189195567


In [6]:
import pprint
pprint.pprint(drift_values)

{'image': None,
 'image.Brightness.mean': {'algorithm': 'ks',
                           'drift_category': 'POSSIBLE_DRIFT',
                           'drift_score': 0.04836288189195567,
                           'pvalue': 0.10439107607489573,
                           'statistic': 0.05400000000000005,
                           'thresholds': {'DRIFT': (0, 0.05),
                                          'NO_DRIFT': (0.15, 1),
                                          'POSSIBLE_DRIFT': (0.05, 0.15)}},
 'image.Brightness.stddev': {'algorithm': 'ks',
                             'drift_category': 'DRIFT',
                             'drift_score': 0.11799981148502028,
                             'pvalue': 1.5975845738597257e-06,
                             'statistic': 0.118,
                             'thresholds': {'DRIFT': (0, 0.05),
                                            'NO_DRIFT': (0.15, 1),
                                            'POSSIBLE_DRIFT': (0.05, 0.15)}},


In [10]:
from whylogs.viz import NotebookProfileVisualizer

# set target and referenece profiles
visualization = NotebookProfileVisualizer()
visualization.set_profiles(target_profile_view=inference_profile, 
                           reference_profile_view= reference_profile)

# generate report
visualization.summary_drift_report()